In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.patches as patches
import matplotlib.cm as cm


from matplotlib.ticker import MaxNLocator
from pathlib import Path
from loguru import logger
logger.remove()

from pim.simulator import SimulationExperiment
from pim.setup import load_results, enumerate_results

In [ ]:
def plot_angular_distances(noise_levels, angular_distances, bins=18, ax=None,
                           label_font_size=11, log_scale=False, title=None):
    fig = None
    if ax is None:
        fig, ax = plt.subplots(subplot_kw=dict(projection='polar'),
                               figsize=(10, 10))

    colors = [cm.viridis(x) for x in np.linspace(0, 1, len(noise_levels))]

    for i in reversed(range(len(noise_levels))):
        plot_angular_distance_histogram(angular_distance=angular_distances[i],
                                        ax=ax, bins=bins, color=colors[i])

    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(22)
    ax.set_title(title, y=1.08, fontsize=label_font_size)

    if log_scale:
        ax.set_rscale('log')
        ax.set_rlim(0.0, 10001)  # What determines this?

    plt.tight_layout()
    return fig, ax


def plot_angular_distance_histogram(angular_distance, ax=None, bins=36,
                                    color='b'):
    fig = None
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))

    radii = np.histogram(angular_distance,
                         np.linspace(-np.pi - np.pi / bins,
                                     np.pi + np.pi / bins,
                                     bins + 2,
                                     endpoint=True))[0]
    radii[0] += radii[-1]
    radii = radii[:-1]
    radii = np.roll(radii, int(bins/2))
    radii = np.append(radii, radii[0])
    # Set all values to have at least a count of 1
    # Need this hack to get the plot fill to work reliably
    radii[radii == 0] = 1
    theta = np.linspace(0, 2 * np.pi, bins+1, endpoint=True)

    ax.plot(theta, radii, color=color, alpha=0.5)
    if color:
        ax.fill_between(theta, 0, radii, alpha=0.2, color=color)
    else:
        ax.fill_between(theta, 0, radii, alpha=0.2)

    return fig, ax

In [ ]:
paths = []
# paths.append("../../results/dye-control_20220919-170804")
# paths.append("../../results/dye-control_20220919-172736")
# paths.append("../../results/dye-control_100-memory")
# paths.append("../../results/dye-control_100-memory-cheat-0.5")
# paths.append("../../results/dye-control_100-memory-2")
# paths.append("../../results/dye-control_100-memory-2-cheat")
# paths.append("../../results/dye-control_100-memory-2-cheat-0.5)
paths.append("../../results/dye-control_100-memory-beta-off")
results = list(load_results(enumerate_results(paths)))


min_dists = np.array([np.linalg.norm(result.closest_position()) for result in results])

mem_errors = np.array([result.memory_error() for result in results])
max_mem_errors = mem_errors.max(axis=0)
min_mem_errors = mem_errors.min(axis=0)
mean_mem_errors = mem_errors.mean(axis=0)

angular_mem_errors = np.array([result.angular_memory_error() for result in results])
max_angular_mem_errors = angular_mem_errors.max(axis=0) * 180/np.pi
min_angular_mem_errors = angular_mem_errors.min(axis=0) * 180/np.pi
mean_angular_mem_errors = angular_mem_errors.mean(axis=0) * 180/np.pi

example = results[3]
timesteps = np.arange(0,3000,1)

fig, ax = plt.subplots(2,2, figsize=(15,15))


# Min dist histogram
confidence = 0.95
interval = np.percentile(min_dists,[100*(1-confidence)/2,100*(1-(1-confidence)/2)])
binwidth = 1

subplot_ax = ax[0,0]
subplot_ax.yaxis.set_major_locator(MaxNLocator(integer=True))
subplot_ax.set_xlabel("Closest distance")
subplot_ax.set_ylabel("Frequency")
subplot_ax.hist(min_dists, bins=np.arange(min(min_dists), max(min_dists) + binwidth, binwidth))
subplot_ax.axvline(interval[1],color="k",linestyle="--")

# Example path
subplot_ax = ax[0,1]
subplot_ax.axis("equal")
subplot_ax.set_title("Example path, 3000 steps")
example.plot_path(ax=subplot_ax, decode=False)

# Memory errors
subplot_ax = ax[1,1]
subplot_ax.set_ylabel("Memory error")
subplot_ax.plot(timesteps, max_mem_errors, color="red", alpha=0.5)
subplot_ax.fill_between(timesteps,max_mem_errors, mean_mem_errors, color="red", alpha=0.2)
subplot_ax.plot(timesteps, mean_mem_errors, color="black")
subplot_ax.fill_between(timesteps,min_mem_errors, mean_mem_errors, color="blue", alpha=0.2)
subplot_ax.plot(timesteps, min_mem_errors, color="blue", alpha=0.5)

# Angular memory errors
subplot_ax = ax[1,0]
# subplot_ax.plot(np.arange(0,3000,1), example.angular_memory_error() * 180/np.pi)
subplot_ax.plot(timesteps, max_angular_mem_errors, color="red", alpha=0.5)
subplot_ax.fill_between(timesteps,max_angular_mem_errors, mean_angular_mem_errors, color="red", alpha=0.2)
subplot_ax.plot(timesteps, mean_angular_mem_errors, color="black")
subplot_ax.fill_between(timesteps,min_angular_mem_errors, mean_angular_mem_errors, color="blue", alpha=0.2)
subplot_ax.plot(timesteps, min_angular_mem_errors, color="blue", alpha=0.5)
subplot_ax.set_ylabel("Angular memory error")
subplot_ax.set_yticks([-180,-90,0,90,180])

In [ ]:
# noise = [0.1,0.2,0.3,0.4]
# angdist = [[2,2],[1,1,1,1,0,0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.7,0.7,0.7],np.ones(35)*np.pi,np.ones(20)*np.pi/2]

plot_angular_distances([1],[angular_mem_errors.flatten()])